In [1]:
import pandas as pd

In [2]:
from bs4 import BeautifulSoup as bs

In [3]:
import requests

In [4]:
url = "https://stats.espncricinfo.com/ci/engine/records/team/match_results.html?id=14450;type=tournament"

In [5]:
page = requests.get(url)

In [6]:
soup = bs(page.text, 'html')

In [8]:
# getting match results table (all rows)
match_results = soup.table.tbody.find_all('tr')

In [9]:
batting_column = ['match', 'team', 'batting_Pos', 'batsman', 'out/not out', 'run', 'ball','m','fours', 'sixes', 'strike_rate']
bolling_column = [ 'match', 'bowlingTeam', 'bowler', 'overs', 'maiden', 'runs', 'wicktes', 'economy', 'zeros', 'fours', 'sixex', 'wides', 'noBalls']
df1 = pd.DataFrame( columns = batting_column )
df2 = pd.DataFrame( columns = bolling_column )

In [12]:
def batting_summary (table, match, team, df1) :
    table = table.tbody.find_all('tr')
    # for batting postion
    index = 1
    for row in table :
        row = row.find_all('td')
        rowi = [match, team, index] + [cell.text.strip() for cell in row]
        
        length = len(df1)
        if len(rowi) >= 8:
            index += 1
            df1.loc[length] = rowi    

In [13]:
def balling_summary (table, match, team, df2) :
    # collect all rows of table
    table = table.tbody.find_all('tr')

    # traverse each row 
    for row in table :
        # from each row fetch all cells
        row = row.find_all('td')
        # add match and team name along with fetched cells to rowi
        rowi = [match, team] + [cell.text.strip() for cell in row]
        
        length = len(df2)
        if len(rowi) >= 8:
            df2.loc[length] = rowi  
    

In [24]:
def crawler ( link, df1, df2 ) :
    # getting resoponse from match summary site
    link_response = requests.get(link)
    link_soup = bs(link_response.text, 'html')

    # fetch team names
    team1 = link_soup.find('div', class_=[ 'ds-mt-3']).find_all('span')[0].text
    team2 = link_soup.find('div', class_=[ 'ds-mt-3']).find_all('span')[2].text

    match = team1 + ' Vs ' + team2

    # batting table for team 1
    table1 = link_soup.find_all('table')[0]
    # balling table for team 1
    table2 = link_soup.find_all('table')[1]
    # batting table for team 2
    table3 = link_soup.find_all('table')[2]
    # batting table for team 2
    table4 = link_soup.find_all('table')[3]

    # pass table1 and table3 to fetch betting details
    batting_summary (table1, match, team1, df1)
    batting_summary (table3, match, team2, df1)

    # pass tab2, tab4 to fetch balling details/summary
    balling_summary (table2, match, team1, df2)
    balling_summary (table4, match, team2, df2)

In [26]:
# Trversseing each row of match results and taking last cell from each row

for row in match_results:
    # fetch last cell from row
    row_data = row.find_all('td')[-1]
    # fetch link from that cell 
    link = 'https://www.espncricinfo.com' + row_data.find('a').get('href')
    # pass fetched link to crawler along with df1, df2
    crawler (link, df1, df2)
    

In [ ]:
## below code is for creating csv 

In [22]:
# df1.to_csv(r"C:\Users\praja\Downloads\shooting-range-master\Projects\T20_2022_players_analysis\web_scrapping/batting_summary.csv")

In [23]:
# df2.to_csv(r"C:\Users\praja\Downloads\shooting-range-master\Projects\T20_2022_players_analysis\web_scrapping/balling_summary.csv")